In [23]:
from pandas import Series, DataFrame
import pandas as pd
from patsy import dmatrices
%pylab inline

# from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler

# to get p-vals
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

# to run log reg and get the coefs
from sklearn.linear_model import LogisticRegression
from sklearn import metrics 


%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [12]:
white_wine = pd.read_csv('wine+quality/winequality-white.csv', sep=';').drop_duplicates()
colNames = ['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar', 'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality']
white_wine.columns = colNames
white_wine

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
6,6.2,0.32,0.16,7.0,0.045,30.0,136.0,0.99490,3.18,0.47,9.6,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [13]:
# change into binary --> good wine has score of 7+, else is bad
white_wine['target'] = white_wine['quality'].apply(lambda x: 1 if x >= 7 else 0)
white_wine['target'].value_counts()

0    3136
1     825
Name: target, dtype: int64

In [14]:
X = white_wine.drop(['quality', 'target'], axis = 1)
y = white_wine['target']


In [15]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [16]:
# Add an intercept term to the scaled features
X_scaled = sm.add_constant(X_scaled)

# Split the data into training and test sets (you can use any other method to split the data)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=1)

# Fit the logistic regression model
logit_model = sm.Logit(y_train, X_train)
logit_result = logit_model.fit()

# Print the summary of the logistic regression model
print(logit_result.summary())


Optimization terminated successfully.
         Current function value: 0.395041
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                 target   No. Observations:                 2772
Model:                          Logit   Df Residuals:                     2760
Method:                           MLE   Df Model:                           11
Date:                Fri, 04 Aug 2023   Pseudo R-squ.:                  0.2369
Time:                        21:54:33   Log-Likelihood:                -1095.1
converged:                       True   LL-Null:                       -1435.1
Covariance Type:            nonrobust   LLR p-value:                1.036e-138
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.8828      0.073    -25.845      0.000      -2.026      -1.740
x1             0.3813      0.

In [17]:
# Get the p-values and corresponding variable names
p_values = logit_result.pvalues[1:]
variable_names = X.columns

# Combine variable names and p-values into a DataFrame for easier interpretation
p_values_df = pd.DataFrame({'Variable': variable_names, 'P-Value': p_values})
print(p_values_df)

                 Variable       P-Value
x1          fixed_acidity  1.640111e-04
x2       volatile_acidity  3.236556e-06
x3            citric_acid  8.289482e-01
x4         residual_sugar  5.141865e-07
x5              chlorides  8.574005e-06
x6    free_sulfur_dioxide  1.737174e-04
x7   total_sulfur_dioxide  8.084902e-02
x8                density  1.698186e-05
x9                     pH  7.661145e-11
x10             sulphates  1.604116e-06
x11               alcohol  1.718325e-02


In [21]:
logreg_model = LogisticRegression()
result = logreg_model.fit(X_train, y_train)

In [24]:
# get accuracy for training
prediction_train_logreg = logreg_model.predict(X_train)
print(metrics.accuracy_score(y_train, prediction_train_logreg))

0.8156565656565656


In [25]:
# accuracy for TESTING
prediction = logreg_model.predict(X_test)
print(metrics.accuracy_score(y_test, prediction))

0.8040370058873002


In [29]:
print('coef: ', logreg_model.coef_)
print()
print('intercept: ', logreg_model.intercept_)

coef:  [[-1.10218455e-04  3.30339795e-01 -3.14963497e-01 -1.60463957e-02
   9.94154760e-01 -5.92479254e-01  2.91201846e-01 -1.71478391e-01
  -1.31984602e+00  5.08673503e-01  2.42144111e-01  5.33342588e-01]]

intercept:  [-1.8731553]


# RED WINE

In [30]:
# load in dataset
red_wine = pd.read_csv('wine+quality/winequality-red.csv', sep=';').drop_duplicates()
colNames = ['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar', 'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality']
red_wine.columns = colNames
red_wine

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
5,7.4,0.660,0.00,1.8,0.075,13.0,40.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1593,6.8,0.620,0.08,1.9,0.068,28.0,38.0,0.99651,3.42,0.82,9.5,6
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [31]:
# change into binary --> good wine has score of 7+, else is bad
red_wine['target'] = red_wine['quality'].apply(lambda x: 1 if x >= 7 else 0)
red_wine['target'].value_counts()

0    1175
1     184
Name: target, dtype: int64

In [33]:
X = red_wine.drop(['quality', 'target'], axis = 1)
y = red_wine['target']

In [34]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [35]:
# Add an intercept term to the scaled features
X_scaled = sm.add_constant(X_scaled)

# Split the data into training and test sets (you can use any other method to split the data)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=1)

# Fit the logistic regression model
logit_model = sm.Logit(y_train, X_train)
logit_result = logit_model.fit()

# Print the summary of the logistic regression model
print(logit_result.summary())


Optimization terminated successfully.
         Current function value: 0.261783
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                 target   No. Observations:                  951
Model:                          Logit   Df Residuals:                      939
Method:                           MLE   Df Model:                           11
Date:                Fri, 04 Aug 2023   Pseudo R-squ.:                  0.3650
Time:                        22:09:07   Log-Likelihood:                -248.96
converged:                       True   LL-Null:                       -392.06
Covariance Type:            nonrobust   LLR p-value:                 6.997e-55
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.1279      0.223    -14.049      0.000      -3.564      -2.692
x1             0.3274      0.

In [36]:
# Get the p-values and corresponding variable names
p_values = logit_result.pvalues[1:]
variable_names = X.columns

# Combine variable names and p-values into a DataFrame for easier interpretation
p_values_df = pd.DataFrame({'Variable': variable_names, 'P-Value': p_values})
print(p_values_df)

                 Variable       P-Value
x1          fixed_acidity  2.486780e-01
x2       volatile_acidity  3.416577e-03
x3            citric_acid  9.127853e-01
x4         residual_sugar  1.100302e-01
x5              chlorides  6.786139e-02
x6    free_sulfur_dioxide  3.077258e-02
x7   total_sulfur_dioxide  9.111546e-05
x8                density  4.898839e-01
x9                     pH  6.513099e-01
x10             sulphates  4.409700e-10
x11               alcohol  6.013860e-08


In [37]:
logreg_model = LogisticRegression()
result = logreg_model.fit(X_train, y_train)

In [38]:
# get accuracy for training
prediction_train_logreg = logreg_model.predict(X_train)
print(metrics.accuracy_score(y_train, prediction_train_logreg))

0.8832807570977917


In [39]:
# accuracy for TESTING
prediction = logreg_model.predict(X_test)
print(metrics.accuracy_score(y_test, prediction))

0.8700980392156863


In [40]:
print('coef: ', logreg_model.coef_)
print()
print('intercept: ', logreg_model.intercept_)

coef:  [[ 7.65198132e-05  3.31867548e-01 -5.62787086e-01 -1.18841988e-03
   2.18868464e-01 -3.20497432e-01  3.49915169e-01 -1.04576903e+00
  -2.09789287e-01 -8.25625837e-02  8.13743005e-01  1.02090966e+00]]

intercept:  [-3.05664252]
